In [1]:
import pandas as pd
import requests
import time
import json
from beeprint import pp
import secrets

In [2]:
# Load data so far
df = pd.read_hdf('pg-text-7-google-books.hdf', 'pg')

In [3]:
df

,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,languages,lcsh,title,...,gb_description,gb_categories,gb_infoLink,gb_pageCount,gb_publishedDate,gb_authors,gb_title,gb_textSnippet,gb_info,gb_oldestBook
0,{},None,None,None,None,{},0.0,None,{},None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{E201, JK}","Jefferson, Thomas",1743,1826,619,{'text/html': 'http://www.gutenberg.org/ebooks...,1.0,[en],"{United States -- History -- Revolution, 1775-...",The Declaration of Independence of the United ...,...,The Declaration of Independence is the usual n...,None,http://books.google.com/books?id=BGq6rQEACAAJ&...,24.0,2014-10-01,[The United States of America Congress Assembled],United States Declaration of Independence,"Since then, it has become a well-known stateme...","{\n ""kind"": ""books#volumes"",\n ""totalItems"": 2...",same
2,"{JK, KF}",United States,None,None,189,{'text/html': 'http://www.gutenberg.org/files/...,2.0,[en],{United States. Constitution. 1st-10th Amendme...,The United States Bill of Rights: The Ten Orig...,...,A collection of 85 articles and essays that we...,[History],https://play.google.com/store/books/details?id...,448.0,2014-07-21,"[Alexander Hamilton, James Madison, John Jay]",The Federalist Papers,A collection of 85 articles and essays that we...,"{\n ""kind"": ""books#volumes"",\n ""totalItems"": 7...",same
3,{E838},"Kennedy, John F. (John Fitzgerald)",1917,1963,36,{'text/html': 'http://www.gutenberg.org/files/...,3.0,[en],{United States -- Foreign relations -- 1961-19...,John F. Kennedy's Inaugural Address,...,"Collected in one illuminating volume, the writ...",[Political Science],http://books.google.com/books?id=XDEhAQAAMAAJ&...,433.0,1991,[John Fitzgerald Kennedy],"""Let the Word Go Forth""",Here are the words that propelled a nation and...,"{\n ""kind"": ""books#volumes"",\n ""totalItems"": 4...",same
4,{E456},"Lincoln, Abraham",1809,1865,66,{'text/html': 'http://www.gutenberg.org/files/...,4.0,[en],{Consecration of cemeteries -- Pennsylvania --...,Lincoln's Gettysburg Address: Given November 1...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"{JK, KF}",United States,None,None,425,{'text/plain': 'http://www.gutenberg.org/ebook...,5.0,[en],"{United States. Constitution, United States --...",The United States Constitution,...,"Google, the most popular search engine worldwi...",[Computers],https://play.google.com/store/books/details?id...,448.0,2004-12-17,[Johnny Long],Google Hacking for Penetration Testers,"This book beats Google hackers to the punch, e...","{\n ""kind"": ""books#volumes"",\n ""totalItems"": 3...",same
6,{E201},"Henry, Patrick",1736,1799,90,{'text/html': 'http://www.gutenberg.org/files/...,6.0,[en],"{Speeches, addresses, etc., American, United S...",Give Me Liberty or Give Me Death,...,"""'Give me Liberty, or give me Death'!"" is a fa...",[Political Science],https://play.google.com/store/books/details?id...,NaN,2015-06-22,[Patrick Henry],Give Me Liberty Or Give Me Death,"&quot;&#39;Give me Liberty, or give me Death&#...","{\n ""kind"": ""books#volumes"",\n ""totalItems"": 7...",same
7,{F001},None,None,None,24,{'text/html': 'http://www.gutenberg.org/files/...,7.0,[en],"{Pilgrims (New Plymouth Colony), Mayflower Com...",The Mayflower Compact,...,None,[Massachusetts],https://play.google.com/store/books/details?id...,NaN,1912,[William Bradford],"History of Plymouth Plantation, 1620-1647",NaN,"{\n ""kind"": ""books#volumes"",\n ""totalItems"": 1...",same
8,{E456},"Lincoln, Abraham",1809,1865,29,{'text/html; charset=us-ascii': 'http://www.gu...,8.0,[en],{United States -- Politics and government -- 1...,Abraham Lincoln's Second Inaugural Address,...,"In the tradition of Wills's ""Lincoln at Gettys...",[History],http://books.google.com/books?id=wtfLtP2LhCcC&...,256.0,2006-11-07,[Ronald C. White],Lincoln's Greatest Speech,In the tradition of Wills&#39;s &quot;Lincoln ...,"{\n ""kind"": ""books#volumes"",\n ""totalItems"": 1...",same
9,{E456},"Lincoln, Abraham",1809,1865,39,{'text

In [6]:
class googleBooks():
    response = False
    parsed = False 
    
    def queryAndParse(self, title, author): 
        self.response = self.queryAPI(title, author)
        if self.response: 
            self.parsed = self.parseResponse(self.response)
        else: 
            return False
        if self.parsed: 
            self.firstBook = self.parsed[0]
            self.getInfo(self.firstBook)
            self.oldestBook = self.getOldestBook(self.parsed)

    def queryAPI(self, title, author): 
        url = 'https://www.googleapis.com/books/v1/volumes'
        if author is "None" or author is None or author == "Various":
            query = "intitle: %s" % title
        else: 
            # Sanitize Author. Just use last name. 
            if "," in author: 
                author = author.split(',')[0]
            query = "intitle: %s inauthor: %s" % (title, author)
        params = [('q', query), ('key', secrets.GB_KEY)]
        response = requests.get(url, params)
        if response.status_code != 200: 
            print('Got a code other than 200! Abandon ship!')
            self.errorCode = response.status_code
            self.errorMsg = response.text
            return False
        else: 
            return response
    
    def parseResponse(self, response): 
        parsed = json.loads(response.text)
        if 'items' in parsed: 
            return parsed['items'] # First book
        else: 
            print("Couldn't find any items.")
            
    def getInfo(self, parsed): 
        self.id = parsed['id']
        self.infoDict = {}
        if 'volumeInfo' in parsed: 
            vi = parsed['volumeInfo']
            for item in ['averageRating', 'ratingsCount', 'description', 'categories', 
                        'infoLink', 'pageCount', 'publishedDate', 'authors', 'title']: 
                if item in vi: 
                    self.infoDict[item] = vi[item]
                else: 
                    self.infoDict[item] = None        
        if 'searchInfo' in parsed: 
            if 'textSnippet' in parsed['searchInfo']: 
                self.infoDict['textSnippet'] = parsed['searchInfo']['textSnippet']
    
    def getOldestBook(self, parsed): 
        pubDates = {}
        for i, item in enumerate(parsed): 
            if 'volumeInfo' in item: 
                if 'publishedDate' in item['volumeInfo']: 
                    pubDates[i] = item['volumeInfo']['publishedDate']
        order = sorted(pubDates, key=pubDates.get)
        oldestBook = parsed[order[0]]
        if oldestBook['id'] == self.id: 
            return 'same'
        else: 
            self.oldestBookDate = oldestBook['volumeInfo']['publishedDate']
            return oldestBook
        
    

In [7]:
title = "The Book of Snobs"
author = "Thackeray, William Makepeace"

In [8]:
gb = googleBooks()
result = gb.queryAndParse(title, author)

In [9]:
# pp(gb.oldestBook)

In [10]:
start, end = 2351, 100000
for i, row in df.iterrows():
    if i >= start and i <= end: 
        gb = googleBooks()
        print('---Looking up #%s: %s' % (i, row.title))
        print('---by %s' % row.author)
        time.sleep(1)
        response = gb.queryAndParse(row.title, row.author)
        if response == False: 
            print('Response false! Abandon ship!')
            break
        if hasattr(gb, 'infoDict'):
            print('+++Found work with title: ', gb.infoDict['title'])
            if 'publishedDate' in gb.infoDict: 
                print('+++Pubdate: %s' % gb.infoDict['publishedDate'])
            for field in ['averageRating', 'ratingsCount', 'description', 'categories', 
                        'infoLink', 'pageCount', 'publishedDate', 'authors', 'title', 'textSnippet']: 
                if field in gb.infoDict: 
                    df.set_value(i, 'gb_'+field, gb.infoDict[field])
            if hasattr(gb, 'response') and hasattr(gb.response, 'text'): 
                df.set_value(i, 'gb_info', gb.response.text)
            if hasattr(gb, 'oldestBook'): 
                df.set_value(i, 'gb_oldestBook', gb.oldestBook)
            if hasattr(gb, 'oldestBookDate'):
                df.set_value(i, 'gb_oldestBookDate', gb.oldestBookDate)
        else: 
            print("XXX Couldn't find it argh!")
        print('')
    
        

---Looking up #2351: His Dog
---by Terhune, Albert Payson
+++Found work with title:  His Dog
+++Pubdate: 1922

---Looking up #2352: The Grand Canyon of Arizona: How to See It
---by James, George Wharton
+++Found work with title:  Twelve Years a Slave
+++Pubdate: 2014-01-22

---Looking up #2353: The Golden Fleece and the Heroes Who Lived Before Achilles
---by Colum, Padraic
+++Found work with title:  The Golden Fleece and the Heroes Who Lived Before Achilles
+++Pubdate: 1921

---Looking up #2354: History of the Catholic Church from the Renaissance to the French Revolution — Volume 1
---by MacCaffrey, James
+++Found work with title:  Atlas of the Historical Geography of the United States
+++Pubdate: 1932

---Looking up #2355: The Story of My Life: With her letters (1887-1901) and a supplementary account of her education, including passages from the reports and letters of her teacher, Anne Mansfield Sullivan, by John Albert Macy
---by Keller, Helen
Couldn't find any items.
XXX Couldn't fi

+++Found work with title:  MLA Handbook for Writers of Research Papers
+++Pubdate: 1999

---Looking up #2403: Letters on England
---by Voltaire
+++Found work with title:  Letters Concerning the English Nation
+++Pubdate: 1778

---Looking up #2404: An Enemy of the People
---by Ibsen, Henrik
+++Found work with title:  An Enemy of the People
+++Pubdate: 1997

---Looking up #2405: Eminent Victorians
---by Strachey, Lytton
+++Found work with title:  Eminent Victorians
+++Pubdate: 2003

---Looking up #2406: Henry Ossian Flipper, the Colored Cadet at West Point: Autobiography of Lieut. Henry Ossian Flipper, U.S.A., First Graduate of Color from the U.S. Military Academy
---by Flipper, Henry Ossian
+++Found work with title:  Twelve Years a Slave
+++Pubdate: 2014-01-22

---Looking up #2407: The Common Law
---by Holmes, Oliver Wendell
+++Found work with title:  The Common Law
+++Pubdate: 1909

---Looking up #2408: Boyhood
---by Tolstoy, Leo, graf
+++Found work with title:  Twelve Years a Slave
++

KeyboardInterrupt: 

In [ ]:
gb.errorMsg

In [ ]:
df.to_hdf('pg-text-7-google-books.hdf', 'pg')